# Deep Learning applied to structured and time series data


## Setup

In [1]:
!pip install pandas-summary
!pip install sklearn-pandas

In [0]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/rossman_project')

In [0]:
from imports import *
from preprocessing_functions import *

In [0]:
HOME_PATH = '/content/gdrive/My Drive/rossman_project/'
DATA_PATH = os.path.join(HOME_PATH, 'data')

## Import dataset

Feature Space:
* train: Training set provided by competition
* store: List of stores
* store_states: mapping of store to the German state they are in
* state_names: list of German state names
* googletrend: trend of certain google keywords over time, found by users to correlate well w/ given data
* weather: weather
* test: testing set

In [0]:
table_names = ['train', 'store', 'store_states', 'state_names', 
               'googletrend', 'weather', 'test']

In [0]:
tables = [pd.read_csv(f'{DATA_PATH}{fname}.csv', low_memory=False) for fname in table_names]

In [0]:
zipped_tables = list(zip(table_names, tables))

## Exporatory data shortcut

In [0]:
for t in zipped_tables: display(f'Table: {t[0]}'), display(t[1].head())

'Table: train'

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


'Table: store'

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


'Table: store_states'

...

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67


'Table: weather'

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,...,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,...,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,...,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,...,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,...,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268


'Table: test'

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [0]:
for t in zipped_tables: display(f'Table: {t[0]}'), display(DataFrameSummary(t[1]).summary())

'Table: train'

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
count,1.01721e+06,1.01721e+06,NaN,1.01721e+06,1.01721e+06,1.01721e+06,1.01721e+06,NaN,1.01721e+06
mean,558.43,3.99834,NaN,5773.82,633.146,0.830107,0.381515,NaN,0.178647
std,321.909,1.99739,NaN,3849.93,464.412,0.375539,0.485759,NaN,0.383056
min,1,1,NaN,0,0,0,0,NaN,0
25%,280,2,NaN,3727,405,1,0,NaN,0
50%,558,4,NaN,5744,609,1,0,NaN,0
75%,838,6,NaN,7856,837,1,1,NaN,0
max,1115,7,NaN,41551,7388,1,1,NaN,1
counts,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209,1017209
uniques,1115,7,942,21734,4086,2,2,4,2


'Table: store'

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1115,NaN,NaN,1112,761,761,1115,571,571,NaN
mean,558,NaN,NaN,5404.9,7.2247,2008.67,0.512108,23.5954,2011.76,NaN
std,322.017,NaN,NaN,7663.17,3.21235,6.19598,0.500078,14.142,1.67494,NaN
min,1,NaN,NaN,20,1,1900,0,1,2009,NaN
25%,279.5,NaN,NaN,717.5,4,2006,0,13,2011,NaN
50%,558,NaN,NaN,2325,8,2010,1,22,2012,NaN
75%,836.5,NaN,NaN,6882.5,10,2013,1,37,2013,NaN
max,1115,NaN,NaN,75860,12,2015,1,50,2015,NaN
counts,1115,1115,1115,1112,761,761,1115,571,571,571
uniques,1115,4,3,654,12,23,2,24,7,3


'Table: store_states'

...

,file,week,trend
count,NaN,NaN,2072
mean,NaN,NaN,63.8142
std,NaN,NaN,12.6502
min,NaN,NaN,0
25%,NaN,NaN,55
50%,NaN,NaN,64
75%,NaN,NaN,72
max,NaN,NaN,100
counts,2072,2072,2072
uniques,14,148,68


'Table: weather'

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,...,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
count,NaN,NaN,15840,15840,15840,15840,15840,15840,15840,15840,...,15459,15459,15459,15840,15840,3604,15840,14667,NaN,15840
mean,NaN,NaN,14.6441,10.389,6.19899,8.58782,6.20581,3.62614,93.6596,74.2829,...,24.0576,12.2398,7.02516,22.7666,11.9722,48.8643,0.831718,5.55131,NaN,175.897
std,NaN,NaN,8.64601,7.37926,6.52639,6.24478,6.08677,6.12839,7.67853,13.4866,...,8.9768,5.06794,4.9806,8.98862,5.87284,13.027,2.51351,1.68771,NaN,101.589
min,NaN,NaN,-11,-13,-15,-14,-15,-73,44,30,...,0,0,0,3,2,21,0,0,NaN,-1
25%,NaN,NaN,8,4,1,4,2,-1,90.75,65,...,14,10,3,16,8,39,0,5,NaN,80
50%,NaN,NaN,15,11,7,9,7,4,94,76,...,31,11,7,21,11,48,0,6,NaN,202
75%,NaN,NaN,21,16,11,13,11,8,100,85,...,31,14,10,27,14,55,0.25,7,NaN,256
max,NaN,NaN,39,31,24,25,20,19,100,100,...,31,31,31,101,53,111,58.93,8,NaN,360
counts,15840,15840,15840,15840,15840,15840,15840,15840,15840,15840,...,15459,15459,15459,15840,15840,3604,15840,14667,11889,15840
uniques,16,990,51,45,40,40,36,40,53,71,...,24,32,24,44,29,47,41,9,21,362


'Table: test'

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
count,41088,41088,41088,NaN,41077,41088,NaN,41088
mean,20544.5,555.9,3.97917,NaN,0.854322,0.395833,NaN,0.443487
std,11861.2,320.274,2.01548,NaN,0.352787,0.489035,NaN,0.496802
min,1,1,1,NaN,0,0,NaN,0
25%,10272.8,279.75,2,NaN,1,0,NaN,0
50%,20544.5,553.5,4,NaN,1,0,NaN,0
75%,30816.2,832.25,6,NaN,1,1,NaN,1
max,41088,1115,7,NaN,1,1,NaN,1
counts,41088,41088,41088,41088,41077,41088,41088,41088
uniques,41088,856,7,48,2,2,2,2


## Data Cleaning / Feature Engineering

As a structured data problem, we necessarily have to go through all the cleaning and feature engineering, even though we're using a neural network.

In [0]:
train, store, store_states, state_names, googletrend, weather, test = tables

In [0]:
len(train),len(test)

(1017209, 41088)

We turn state Holidays to booleans, to make them more convenient for modeling. We can do calculations on pandas fields using notation very similar (often identical) to numpy.

In [0]:
train.StateHoliday = train.StateHoliday!='0'
test.StateHoliday = test.StateHoliday!='0'

`join_df` is a function for joining tables on specific fields. By default, we'll be doing a left outer join of `right` on the `left` argument using the given fields for each table.

Pandas does joins using the `merge` method. The `suffixes` argument describes the naming convention for duplicate fields. We've elected to leave the duplicate field names on the left untouched, and append a "\_y" to those on the right.

In [0]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, suffixes=("", suffix))

Join weather/state names:

In [0]:
table_names

['train',
 'store',
 'store_states',
 'state_names',
 'googletrend',
 'weather',
 'test']

In [0]:
weather = join_df(weather, state_names, "file", "StateName")

In [0]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'

The following extracts particular date fields from a complete datetime for the purpose of constructing categoricals.

You should *always* consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities. We'll add to every table with a date field.

In [0]:
add_datepart(weather, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)

The Google trends data has a special category for the whole of Germany - we'll pull that out so we can use it explicitly.

In [0]:
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

In [0]:
store = join_df(store, store_states, "Store")
len(store[store.State.isnull()])

0

In [0]:
joined = join_df(train, store, "Store")
joined_test = join_df(test, store, "Store")
len(joined[joined.StoreType.isnull()]),len(joined_test[joined_test.StoreType.isnull()])

(0, 0)

In [0]:
joined = join_df(joined, googletrend, ["State","Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()]),len(joined_test[joined_test.trend.isnull()])

(0, 0)

In [0]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()]),len(joined_test[joined_test.trend_DE.isnull()])

(0, 0)

In [0]:
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()]),len(joined_test[joined_test.Mean_TemperatureC.isnull()])

(0, 0)

In [0]:
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns:
                df.drop(c, inplace=True, axis=1)

Next we'll fill in missing values to avoid complications with `NA`'s. 

In [0]:
for df in (joined,joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

Next we'll extract features "CompetitionOpenSince" and "CompetitionDaysOpen".

In [0]:
for df in (joined,joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

In [0]:
joined.head()

We'll replace some erroneous / outlying data.

In [0]:
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

We add "CompetitionMonthsOpen" field, limiting the maximum to 2 years to limit number of unique categories.

In [0]:
for df in (joined,joined_test):
    df["CompetitionMonthsOpen"] = df["CompetitionDaysOpen"]//30
    df.loc[df.CompetitionMonthsOpen>24, "CompetitionMonthsOpen"] = 24
joined.CompetitionMonthsOpen.unique()

Same process for Promo dates.

In [0]:
 df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))

In [0]:
for df in (joined,joined_test):
    df['_Month'] = (df.Promo2SinceWeek*7 - 6) // 30 + 1
    df.loc[df._Month>12, "_Month"] = 12
    df.loc[df._Month<1, "_Month"] = 1
    df['_Day'] = (df.Promo2SinceWeek*7 - 6) - (df['_Month'] - 1)*30
    df.loc[df._Day>28, "_Day"] = 28
    df.loc[df._Day<1, "_Day"] = 1
    df['Promo2Since'] = pd.to_datetime(dict(year=df.Promo2SinceYear, month=df._Month, day=df._Day))
    df["Promo2Days"] = df.Date.subtract(df["Promo2Since"]).dt.days
    
    del df['_Month']
    del df['_Day']

In [0]:
for df in (joined,joined_test):
    df.loc[df.Promo2Days<0, "Promo2Days"] = 0
    df.loc[df.Promo2SinceYear<1990, "Promo2Days"] = 0
    df["Promo2Weeks"] = df["Promo2Days"]//7
    df.loc[df.Promo2Weeks<0, "Promo2Weeks"] = 0
    df.loc[df.Promo2Weeks>25, "Promo2Weeks"] = 25
    df.Promo2Weeks.unique()

In [0]:
joined.loc[0:5, ['Date','Promo', 'Promo2', 'Promo2SinceYear', 'Promo2SinceWeek', 'Promo2Since', 'Promo2Days']]

In [0]:
joined.to_pickle(f'{DATA_PATH}joined.pkl')
joined_test.to_pickle(f'{DATA_PATH}joined_test.pkl')

## Durations

It is common when working with time series data to extract data that explains relationships across rows as opposed to columns, e.g.:
* Running averages
* Time until next event
* Time since last event

This is often difficult to do with most table manipulation frameworks, since they are designed to work with relationships across columns. As such, we've created a class to handle this type of data.

We'll define a function `get_elapsed` for cumulative counting across a sorted dataframe. Given a particular field `fld` to monitor, this function will start tracking time since the last occurrence of that field. When the field is seen again, the counter is set to zero.

Upon initialization, this will result in datetime na's until the field is encountered. This is reset every time a new store is seen. We'll see how to use this shortly.

In [0]:
df.sort_values(['Store', 'Date']).head(10)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,Week_y
1016095,1,2,2013-01-01,0,0,0,0,True,1,2013,...,10.0,23,14,39.0,2.03,6.0,Rain,206,Hessen,1
1014980,1,3,2013-01-02,5530,668,1,0,False,1,2013,...,10.0,19,11,37.0,0.00,6.0,NaN,200,Hessen,1
1013865,1,4,2013-01-03,4327,578,1,0,False,1,2013,...,6.0,29,19,47.0,0.00,7.0,Rain,191,Hessen,1
1012750,1,5,2013-01-04,4486,619,1,0,False,1,2013,...,6.0,21,13,37.0,0.00,6.0,Rain,210,Hessen,1
1011635,1,6,2013-01-05,4997,635,1,0,False,1,2013,...,10.0,24,18,NaN,0.00,7.0,Rain,281,Hessen,1
1010520,1,7,2013-01-06,0,0,0,0,False,1,2013,...,10.0,21,13,NaN,0.00,7.0,Rain,259,Hessen,1
1009405,1,1,2013-01-07,7176,785,1,1,False,1,2013,...,10.0,14,8,NaN,0.00,6.0,Rain,235,Hessen,2
1008290,1,2,2013-01-08,5580,654,1,1,False,1,2013,...,3.0,16,10,NaN,0.00,7.0,Rain,209,Hessen,2
1007175,1,3,2013-01-09,5471,626,1,1,False,1,2013,...,5.0,19,13,29.0,0.76,7.0,Rain,207,Hessen,2
1006060,1,4,2013-01-10,4892,615,1,1,False,1,2013,...,6.0,21,13,34.0,1.02,6.0,Rain,208,Hessen,2


We'll be applying this to a subset of columns:

In [0]:
columns = ["Date", "Store", "Promo", "StateHoliday", "SchoolHoliday"]

In [0]:
df = train[columns].append(test[columns])

Let's walk through an example.

Say we're looking at School Holiday. We'll first sort by Store, then Date, and then call `add_elapsed('SchoolHoliday', 'After')`:
This will apply to each row with School Holiday:
* A applied to every row of the dataframe in order of store and date
* Will add to the dataframe the days since seeing a School Holiday
* If we sort in the other direction, this will count the days until another holiday.

In [0]:
df.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,...,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName
40232,40233,1,6,2015-08-01,1.0,0,False,1,2015,8,...,NaN,NaN,23,14,NaN,0.0,NaN,NaN,55,Hessen
39376,39377,1,7,2015-08-02,0.0,0,False,1,2015,8,...,NaN,NaN,16,6,NaN,0.0,NaN,NaN,55,Hessen
38520,38521,1,1,2015-08-03,1.0,1,False,1,2015,8,...,NaN,NaN,23,10,40.0,0.0,NaN,NaN,119,Hessen
37664,37665,1,2,2015-08-04,1.0,1,False,1,2015,8,...,11.0,7.0,40,11,NaN,6.1,6.0,Rain,281,Hessen
36808,36809,1,3,2015-08-05,1.0,1,False,1,2015,8,...,11.0,10.0,16,6,NaN,0.0,1.0,Fog,60,Hessen


In [0]:
fld = 'SchoolHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(df, 'Store', 'Date', fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(df, 'Store', 'Date', fld, 'Before')

We'll do this for two more fields.

In [0]:
fld = 'StateHoliday'
df = df.sort_values(['Store', 'Date'])
get_elapsed(df, 'Store', 'Date', fld, 'After')
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
get_elapsed(df, 'Store', 'Date', fld, 'Before')

In [0]:
fld = 'Promo'
df = df.sort_values(['Store', 'Date'])
get_elapsed(df, 'Store', 'Date', fld, 'After')
df = df.sort_values(['Store',  'Date'], ascending=[True, False])
get_elapsed(df, 'Store', 'Date',fld, 'Before')

We're going to set the active index to Date.

In [0]:
df = df.set_index("Date")

Then set null values from elapsed field calculations to 0.

In [0]:
df.head()

,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Week,...,CloudCover,Events,WindDirDegrees,StateName,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
Date,,,,,,,,,,,,,,,,,,,,,
2015-09-17,1,1,4,1.0,1,False,0,2015,9,38,...,6.0,Rain,191,Hessen,13.0,NaN,NaN,NaN,0.0,0.0
2015-09-16,857,1,3,1.0,1,False,0,2015,9,38,...,7.0,Fog-Rain-Thunderstorm,41,Hessen,12.0,NaN,NaN,NaN,0.0,0.0
2015-09-15,1713,1,2,1.0,1,False,0,2015,9,38,...,5.0,Rain,180,Hessen,11.0,NaN,NaN,NaN,0.0,0.0
2015-09-14,2569,1,1,1.0,1,False,0,2015,9,38,...,5.0,Rain-Thunderstorm,188,Hessen,10.0,NaN,NaN,NaN,0.0,0.0
2015-09-13,3425,1,7,0.0,0,False,0,2015,9,37,...,7.0,Fog-Rain,83,Hessen,9.0,NaN,NaN,NaN,9.0,-1.0


In [0]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [0]:
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

Next we'll demonstrate window functions in pandas to calculate rolling quantities.

Here we're sorting by date (`sort_index()`) and counting the number of events of interest (`sum()`) defined in `columns` in the following week (`rolling()`), grouped by Store (`groupby()`). We do the same in the opposite direction.

In [0]:
df.head()

,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Week,...,CloudCover,Events,WindDirDegrees,StateName,AfterSchoolHoliday,BeforeSchoolHoliday,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo
Date,,,,,,,,,,,,,,,,,,,,,
2015-09-17,1,1,4,1.0,1,False,0,2015,9,38,...,6.0,Rain,191,Hessen,13,0,0,0,0,0
2015-09-16,857,1,3,1.0,1,False,0,2015,9,38,...,7.0,Fog-Rain-Thunderstorm,41,Hessen,12,0,0,0,0,0
2015-09-15,1713,1,2,1.0,1,False,0,2015,9,38,...,5.0,Rain,180,Hessen,11,0,0,0,0,0
2015-09-14,2569,1,1,1.0,1,False,0,2015,9,38,...,5.0,Rain-Thunderstorm,188,Hessen,10,0,0,0,0,0
2015-09-13,3425,1,7,0.0,0,False,0,2015,9,37,...,7.0,Fog-Rain,83,Hessen,9,0,0,0,9,-1


In [0]:
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

In [0]:
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()

Next we want to drop the Store indices grouped together in the window function.

Often in pandas, there is an option to do this in place. This is time and memory efficient when working with large datasets.

In [0]:
bwd.drop('Store',1,inplace=True)
bwd.reset_index(inplace=True)

In [0]:
fwd.drop('Store',1,inplace=True)
fwd.reset_index(inplace=True)

In [0]:
df.reset_index(inplace=True)

Now we'll merge these values onto the df.

In [0]:
df = df.merge(bwd, 'left', ['Date', 'Store'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['Date', 'Store'], suffixes=['', '_fw'])

In [0]:
columns

['SchoolHoliday', 'StateHoliday', 'Promo']

In [0]:
df.drop(columns,1,inplace=True)

In [0]:
df.head()

,Date,Id,Store,DayOfWeek,Open,Year,Month,Week,Day,Dayofweek,...,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,2015-09-17,1,1,4,1.0,2015,9,38,17,3,...,0,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
1,2015-09-16,857,1,3,1.0,2015,9,38,16,2,...,0,0,0,0,0.0,0.0,3.0,0.0,0.0,2.0
2,2015-09-15,1713,1,2,1.0,2015,9,38,15,1,...,0,0,0,0,0.0,0.0,2.0,0.0,0.0,3.0
3,2015-09-14,2569,1,1,1.0,2015,9,38,14,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,4.0
4,2015-09-13,3425,1,7,0.0,2015,9,37,13,6,...,0,0,9,-1,0.0,0.0,0.0,0.0,0.0,4.0


It's usually a good idea to back up large tables of extracted / wrangled features before you join them onto another one, that way you can go back to it easily if you need to make changes to it.

In [0]:
df.to_pickle(os.path.join(DATA_PATH, 'df.pkl'))

In [0]:
df = pd.read_pickle(os.path.join(DATA_PATH, 'df.pkl'))

In [0]:
df.head()

,Date,Id,Store,DayOfWeek,Open,Year,Month,Week,Day,Dayofweek,...,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,2015-09-17,1,1,4,1.0,2015,9,38,17,3,...,0,0,0,0,0.0,0.0,4.0,0.0,0.0,1.0
1,2015-09-16,857,1,3,1.0,2015,9,38,16,2,...,0,0,0,0,0.0,0.0,3.0,0.0,0.0,2.0
2,2015-09-15,1713,1,2,1.0,2015,9,38,15,1,...,0,0,0,0,0.0,0.0,2.0,0.0,0.0,3.0
3,2015-09-14,2569,1,1,1.0,2015,9,38,14,0,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,4.0
4,2015-09-13,3425,1,7,0.0,2015,9,37,13,6,...,0,0,9,-1,0.0,0.0,0.0,0.0,0.0,4.0


In [0]:
df["Date"] = pd.to_datetime(df.Date)

In [0]:
df.columns

Index(['Date', 'Id', 'Store', 'DayOfWeek', 'Open', 'Year', 'Month', 'Week',
       'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start',
       'Elapsed', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State', 'file',
       'week', 'trend', 'file_DE', 'week_DE', 'trend_DE', 'Date_DE',
       'State_DE', 'Month_DE', 'Day_DE', 'Dayofweek_DE', 'Dayofyear_DE',
       'Is_month_end_DE', 'Is_month_start_DE', 'Is_quarter_end_DE',
       'Is_quarter_start_DE', 'Is_year_end_DE', 'Is_year_start_DE',
       'Elapsed_DE', 'Max_TemperatureC', 'Mean_TemperatureC',
       'Min_TemperatureC', 'Dew_PointC', 'MeanDew_PointC', 'Min_DewpointC',
       'Max_Humidity', 'Mean_Humidity', 'Min_Humidity',
       'Max_Sea_Level_PressurehPa', 'Mean_Sea_Level_PressurehPa',
       'Min_Se

In [0]:
joined = join_df(joined, df, ['Store', 'Date'])

In [0]:
joined_test = join_df(joined_test, df, ['Store', 'Date'])

The authors also removed all instances where the store had zero sale / was closed. We speculate that this may have cost them a higher standing in the competition. One reason this may be the case is that a little exploratory data analysis reveals that there are often periods where stores are closed, typically for refurbishment. Before and after these periods, there are naturally spikes in sales that one might expect. By ommitting this data from their training, the authors gave up the ability to leverage information about these periods to predict this otherwise volatile behavior.

In [0]:
joined = joined[joined.Sales!=0]

We'll back this up as well.

In [0]:
joined.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,AfterStateHoliday,BeforeStateHoliday,AfterPromo,BeforePromo,SchoolHoliday_bw,StateHoliday_bw,Promo_bw,SchoolHoliday_fw,StateHoliday_fw,Promo_fw
0,1,5,2015-07-31,5263,555,1,1,False,1,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,False,1,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,5,2015-07-31,8314,821,1,1,False,1,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,5,2015-07-31,13995,1498,1,1,False,1,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,False,1,2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
joined.reset_index(inplace=True)
joined_test.reset_index(inplace=True)

In [0]:
joined.to_pickle(os.path.join(DATA_PATH, 'joined.pkl'))
joined_test.to_pickle(os.path.join(DATA_PATH, 'joined_test.pkl'))

We now have our final set of engineered features.

While these steps were explicitly outlined in the paper, these are all fairly typical feature engineering steps for dealing with time series data and are practical in any similar setting.

## Create features

In [0]:
joined = pd.read_pickle(os.path.join(DATA_PATH, 'joined.pkl'))
joined_test = pd.read_pickle(os.path.join(DATA_PATH, 'joined_test.pkl'))

In [8]:
joined.head().T.head(20)

,0,1,2,3,4
index,0,1,2,3,4
Store,1,2,3,4,5
DayOfWeek,5,5,5,5,5
Date,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00,2015-07-31 00:00:00
Sales,5263,6064,8314,13995,4822
Customers,555,625,821,1498,559
Open,1,1,1,1,1
Promo,1,1,1,1,1
StateHoliday,False,False,False,False,False
SchoolHoliday,1,1,1,1,1


Now that we've engineered all our features, we need to convert to input compatible with a neural network.

This includes converting categorical variables into contiguous integers or one-hot encodings, normalizing continuous features to standard normal, etc...

In [9]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

contin_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

n = len(joined); n

844338

In [0]:
dep = 'Sales'
joined = joined[cat_vars+contin_vars+[dep, 'Date']].copy()

In [0]:
joined_test[dep] = 0
joined_test = joined_test[cat_vars+contin_vars+[dep, 'Date', 'Id']].copy()

In [0]:
for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()

In [0]:
apply_cats(joined_test, joined)

In [0]:
for v in contin_vars:
    joined[v] = joined[v].fillna(0).astype('float32')
    joined_test[v] = joined_test[v].fillna(0).astype('float32')

We can now process our data...

In [0]:
joined = joined.set_index("Date")

In [0]:
df_train, y, nas, mapper = proc_df(joined, 'Sales', do_scale=True)

In [0]:
joined_test = joined_test.set_index("Date")

In [0]:
df_test, _, nas, mapper = proc_df(joined_test, 'Sales', do_scale=True, skip_flds=['Id'],
                                  mapper=mapper, na_dict=nas)

In [0]:
#df_train.to_pickle(os.path.join(DATA_PATH, 'df_train.pkl'))
#y.dump(os.path.join(DATA_PATH, 'y.pkl'))
#df_test.to_pickle(os.path.join(DATA_PATH, 'df_test.pkl'))

In [0]:
#df_train = pd.read_pickle(os.path.join(DATA_PATH, 'df_train.pkl'))
#y = pd.read_pickle(os.path.join(DATA_PATH, 'y.pkl'))

#df_test = pd.read_pickle(os.path.join(DATA_PATH, 'df_test.pkl'))

Transform response variable to scaled log

In [0]:
max_y_log = max(np.log(y))

def yl_for_fit(y):
    return np.log(y) / max_y_log

def inv_yl_for_pred(yl):
    return np.exp(yl * max_y_log)

In [0]:
yl = yl_for_fit(y)

In [21]:
max_y_log

10.634676867382668

### Validation set

Good approach to take **validation set** is using the exact same length of time period as the test set uses - this is implemented here:

In [0]:
val_idx = np.flatnonzero(
    (df_train.index<=pd.datetime(2014,9,17)) & (df_train.index>=pd.datetime(2014,8,1)))

In [23]:
val_idx

array([277141, 277142, 277143, ..., 315537, 315538, 315539])

In [0]:
trn, val, yl_trn, yl_val = split_by_idx(val_idx, df_train, yl)

## Fast-check model

We're ready to put together our models.

Root-mean-squared percent error is the metric Kaggle used for this competition.

In [0]:
from sklearn.ensemble import RandomForestRegressor

In [0]:
def exp_rmspe(y_true, y_pred):
    y_true = inv_yl_for_pred(y_true)
    pct_var = (y_true - inv_yl_for_pred(y_pred))/y_true
    return math.sqrt((pct_var**2).mean())
  

def generate_sub_file(pred, sub_name):
  
  sub = joined_test[['Id']]
  sub.loc[:, 'Sales'] = pred
  sub.to_csv(os.path.join(HOME_PATH, 'submissions',(f'{sub_name}' + '.csv')), index=False) 

In [0]:
rf = RandomForestRegressor(n_estimators=40, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)

In [0]:
rf.fit(trn, yl_trn)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.99, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=-1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [0]:
pred_val = rf.predict(val)

In [0]:
rf.score(trn, yl_trn), rf.score(val, yl_val), rf.oob_score_, exp_rmspe(yl_val,pred_val)

(0.9821932056503099,
 0.9310961778099387,
 0.9242068186561262,
 0.10912028859739947)

In [0]:
## Test kaggle Score
pred_test_l = rf.predict(df_test)
pred_test = inv_yl_for_pred(pred_test_l)

In [0]:
generate_sub_file(pred_test, 'sub_rf')

Kaggle Leaderboard score: 0.127 

## Embeddings

In [0]:
cat_sz = [(c, len(joined[c].cat.categories) + 1) for c in cat_vars]

In [28]:
cat_sz

[('Store', 1116),
 ('DayOfWeek', 8),
 ('Year', 4),
 ('Month', 13),
 ('Day', 32),
 ('StateHoliday', 3),
 ('CompetitionMonthsOpen', 26),
 ('Promo2Weeks', 27),
 ('StoreType', 5),
 ('Assortment', 4),
 ('PromoInterval', 4),
 ('CompetitionOpenSinceYear', 24),
 ('Promo2SinceYear', 9),
 ('State', 13),
 ('Week', 53),
 ('Events', 22),
 ('Promo_fw', 7),
 ('Promo_bw', 7),
 ('StateHoliday_fw', 4),
 ('StateHoliday_bw', 4),
 ('SchoolHoliday_fw', 9),
 ('SchoolHoliday_bw', 9)]

We use the *cardinality* of each variable (that is, its number of unique values) to decide how large to make its *embeddings*. Each level will be associated with a vector with length defined as below.

In [0]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [30]:
emb_szs

[(1116, 50),
 (8, 4),
 (4, 2),
 (13, 7),
 (32, 16),
 (3, 2),
 (26, 13),
 (27, 14),
 (5, 3),
 (4, 2),
 (4, 2),
 (24, 12),
 (9, 5),
 (13, 7),
 (53, 27),
 (22, 11),
 (7, 4),
 (7, 4),
 (4, 2),
 (4, 2),
 (9, 5),
 (9, 5)]

### Simple example of embeddings

Example how embeddings are constructed for one variable DayOfWeek

In [31]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Concatenate, Embedding, Reshape, Dropout
from keras.callbacks import CSVLogger
from keras import optimizers, losses

Using TensorFlow backend.


In [0]:
nam_unique_cat = emb_szs[1][0]
emb_sz = emb_szs[1][1]

inputs = []
embeddings = []    

      
input_shape = Input(shape=(1,))
embedding = Embedding(nam_unique_cat, emb_sz, input_length=1)(input_shape)
embedding = Reshape(target_shape=(emb_sz,))(embedding)
inputs.append(input_shape)
embeddings.append(embedding)
        
model = Model(inputs, embeddings)
model.compile(loss='mean_absolute_error', optimizer='adam')        
  

In [0]:
input_dayofweek = df_train['DayOfWeek'].values

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_47 (InputLayer)        (None, 1)                 0         
_________________________________________________________________
embedding_48 (Embedding)     (None, 1, 4)              32        
_________________________________________________________________
reshape_46 (Reshape)         (None, 4)                 0         
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________


In [0]:
dow_embeddings = model.predict(input_dayofweek)

In [0]:
dow_embeddings

array([[ 0.02596543, -0.04292766, -0.00709865,  0.03921613],
       [ 0.02596543, -0.04292766, -0.00709865,  0.03921613],
       [ 0.02596543, -0.04292766, -0.00709865,  0.03921613],
       ...,
       [ 0.04955788, -0.04527954, -0.0347888 ,  0.00989907],
       [ 0.04955788, -0.04527954, -0.0347888 ,  0.00989907],
       [ 0.04955788, -0.04527954, -0.0347888 ,  0.00989907]],
      dtype=float32)

In [0]:
dow_embeddings.shape

(844338, 4)

Now each categorical variable (here: day of week) is represented by vector of length 4

### Build embedding network for rossman sales prediction


In [0]:
import keras.backend as K

  
def exp_rmspe_k(y_true, y_pred):
    max_y_log = 10.634677
    y_true = K.exp(y_true * max_y_log)
    pct_var = (y_true - K.exp(y_pred * max_y_log))/y_true
    return K.sqrt(K.mean(pct_var**2))

In [0]:
def build_embedding_network(emb_szs):


    inputs = []
    embeddings = []

    for nam_unique_cat, emb_sz in emb_szs:
      
        input_shape = Input(shape=(1,))
        embedding = Embedding(nam_unique_cat, emb_sz, input_length=1)(input_shape)
        embedding = Reshape(target_shape=(emb_sz,))(embedding)
        inputs.append(input_shape)
        embeddings.append(embedding)
        
        
        

    input_distance = Input(shape=(1,))
    embedding_numeric = Dense(1)(input_distance) 
    inputs.append(input_distance)
    embeddings.append(embedding_numeric)

    input_temperature = Input(shape=(3,))
    embedding_numeric = Dense(3)(input_temperature) 
    inputs.append(input_temperature)
    embeddings.append(embedding_numeric)    

    input_humidity = Input(shape=(3,))
    embedding_numeric = Dense(3)(input_humidity) 
    inputs.append(input_humidity)
    embeddings.append(embedding_numeric)  

    input_wind = Input(shape=(2,))
    embedding_numeric = Dense(2)(input_wind) 
    inputs.append(input_wind)
    embeddings.append(embedding_numeric)     

    input_cloud = Input(shape=(1,))
    embedding_numeric = Dense(1)(input_cloud) 
    inputs.append(input_cloud)
    embeddings.append(embedding_numeric)     

    input_trend = Input(shape=(1,))
    embedding_numeric = Dense(1)(input_trend) 
    inputs.append(input_trend)
    embeddings.append(embedding_numeric)      


    input_trend_de = Input(shape=(1,))
    embedding_numeric = Dense(1)(input_trend_de) 
    inputs.append(input_trend_de)
    embeddings.append(embedding_numeric)     

    input_afterstateholiday  = Input(shape=(1,))
    embedding_numeric = Dense(1)(input_afterstateholiday) 
    inputs.append(input_afterstateholiday)
    embeddings.append(embedding_numeric)

    input_beforestateholiday  = Input(shape=(1,))
    embedding_numeric = Dense(1)(input_beforestateholiday) 
    inputs.append(input_beforestateholiday)
    embeddings.append(embedding_numeric)        

    input_promo  = Input(shape=(1,))
    embedding_numeric = Dense(1)(input_promo) 
    inputs.append(input_promo)
    embeddings.append(embedding_numeric)   

    input_schoolholiday  = Input(shape=(1,))
    embedding_numeric = Dense(1)(input_schoolholiday) 
    inputs.append(input_schoolholiday)
    embeddings.append(embedding_numeric)   
        
        
    x = Concatenate()(embeddings)
    x = Dropout(0.02)(x)
    x = Dense(1000, activation='relu', 
              kernel_initializer='random_uniform', bias_initializer='random_uniform')(x)
    x = Dense(500, activation='relu', 
              kernel_initializer='random_uniform', bias_initializer='random_uniform')(x)
    output = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, output)
    model.compile(loss='mean_absolute_error', optimizer=optimizers.Adam(lr=1e-3), metrics=[exp_rmspe_k])

    return model

In [0]:
def convert_df_to_nn_format(df):
  
    res = []
    # categorical vars of shape=(1,)
    for c in df.columns:
      if len(res) < 22:
        res.append(df[c].values)
    
    
    # input_distance
    res.append(df.iloc[:, 22].values)
    # input_temperature
    res.append(df.iloc[:, 23:26].values)    
    # input_humidity
    res.append(df.iloc[:, 26:29].values)      
    # input_wind
    res.append(df.iloc[:, 29:31].values)   
    # input_cloud
    res.append(df.iloc[:, 31].values) 
    # input_trend
    res.append(df.iloc[:, 32].values) 
    # input_trend_de
    res.append(df.iloc[:, 33].values) 
    # input_afterstateholiday
    res.append(df.iloc[:, 34].values) 
    # input_beforestateholiday
    res.append(df.iloc[:, 35].values)   
    # input_promo
    res.append(df.iloc[:, 36].values)     
    # input_schoolholiday
    res.append(df.iloc[:, 37].values)   
            
    return res     

In [0]:
def save_model(model, model_name):
    model.save(os.path.join(HOME_PATH, 'models',(f'{model_name}' + '.h5')))
    
def load_model(model_name):
    return models.load_model(os.path.join(HOME_PATH, 'models',(f'{model_name}' + '.h5')))

In [0]:
def make_prediction(model, df):
  
    list_df = convert_df_to_nn_format(df)
    pred_yl = model.predict(list_df)
    pred_y = inv_yl_for_pred(pred_yl)
    
    return pred_y
  

  
def display_loss_acc_curve(history):
    
    performance = history['exp_rmspe_k']
    val_performance = history['val_exp_rmspe_k']
    epochs = range(1, len(performance) + 1)

    loss = history['loss']
    val_loss = history['val_loss']
    
    fig, ax = plt.subplots(1,2, figsize=(18,6))
    ax[0].plot(epochs, performance, 'bo', label='Training performance')
    ax[0].plot(epochs, val_performance, 'orange', label='Validation performance')
    ax[0].set_title('Training and validation performance')
    ax[0].set_xlabel('Epochs')
    ax[0].set_ylabel('Performance')
    ax[0].legend()

    #fig, ax = plt.subplots(figsize=(8,5))
    ax[1].plot(epochs, loss, 'bo', label='Training loss')
    ax[1].plot(epochs, val_loss, 'orange', label='Validation loss')
    ax[1].set_title('Training and validation loss')
    ax[1].set_xlabel('Epochs')
    ax[1].set_ylabel('loss')
    ax[1].legend()


    plt.show()

In [0]:
NN = build_embedding_network(emb_szs)

In [0]:
NN.summary()

In [0]:
list_trn = convert_df_to_nn_format(trn)
list_val = convert_df_to_nn_format(val)

In [0]:
list_train = convert_df_to_nn_format(df_train)

In [0]:
history = NN.fit(list_trn, yl_trn, epochs=3, batch_size=128, validation_data=(list_val, yl_val))

Train on 805939 samples, validate on 38399 samples
Epoch 1/3
805939/805939 [==============================] - 142s 176us/step - loss: 0.0100 - exp_rmspe_k: 0.1658 - val_loss: 0.0104 - val_exp_rmspe_k: 0.1312
Epoch 2/3
805939/805939 [==============================] - 137s 170us/step - loss: 0.0074 - exp_rmspe_k: 0.1193 - val_loss: 0.0103 - val_exp_rmspe_k: 0.1336
Epoch 3/3
805939/805939 [==============================] - 137s 170us/step - loss: 0.0069 - exp_rmspe_k: 0.1113 - val_loss: 0.0107 - val_exp_rmspe_k: 0.1337


In [0]:
display_loss_acc_curve(history.history)

In [0]:
save_model(NN, 'NN_trn_linear_epochs10_batch_size128')

In [0]:
y_test = make_prediction(NN, df_test)

In [0]:
generate_sub_file(y_test, 'sub_trn_epochs3_batch_size128')

### Build 7 NN and average results for better predictions

In [82]:
for i in range(7):
  
  NN = build_embedding_network(emb_szs)
  
  NN.fit(list_train, yl, epochs=20, batch_size=128)
  
  model_name = f'NN_trn_epochs20_batch_size128_iter_{i}'  
  save_model(NN, model_name)

  y_test = make_prediction(NN, df_test)
  
  sub_name = f'sub__epochs20_batch_size128_{i}'
  generate_sub_file(y_test, sub_name)
  
  
  print(f'Finished iteration {i}')
  

Epoch 1/20
844338/844338 [==============================] - 134s 158us/step - loss: 0.0099 - exp_rmspe_k: 0.1609
Epoch 2/20
844338/844338 [==============================] - 132s 156us/step - loss: 0.0074 - exp_rmspe_k: 0.1195
Epoch 3/20
844338/844338 [==============================] - 132s 156us/step - loss: 0.0069 - exp_rmspe_k: 0.1105
Epoch 4/20
844338/844338 [==============================] - 132s 156us/step - loss: 0.0066 - exp_rmspe_k: 0.1061
Epoch 5/20
844338/844338 [==============================] - 131s 155us/step - loss: 0.0064 - exp_rmspe_k: 0.1030
Epoch 6/20
844338/844338 [==============================] - 131s 155us/step - loss: 0.0062 - exp_rmspe_k: 0.1009
Epoch 7/20
844338/844338 [==============================] - 131s 155us/step - loss: 0.0061 - exp_rmspe_k: 0.0993
Epoch 8/20
844338/844338 [==============================] - 131s 155us/step - loss: 0.0060 - exp_rmspe_k: 0.0976
Epoch 9/20
844338/844338 [==============================] - 132s 156us/step - loss: 0.0059 - exp

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Finished iteration 7
Epoch 1/20
844338/844338 [==============================] - 134s 158us/step - loss: 0.0099 - exp_rmspe_k: 0.1625
Epoch 2/20
844338/844338 [==============================] - 132s 157us/step - loss: 0.0074 - exp_rmspe_k: 0.1190
Epoch 3/20
844338/844338 [==============================] - 133s 157us/step - loss: 0.0069 - exp_rmspe_k: 0.1103
Epoch 4/20
844338/844338 [==============================] - 133s 157us/step - loss: 0.0066 - exp_rmspe_k: 0.1062
Epoch 5/20
844338/844338 [==============================] - 133s 158us/step - loss: 0.0064 - exp_rmspe_k: 0.1025
Epoch 6/20
844338/844338 [==============================] - 132s 157us/step - loss: 0.0062 - exp_rmspe_k: 0.1000
Epoch 7/20
844338/844338 [==============================] - 132s 157us/step - loss: 0.0061 - exp_rmspe_k: 0.0984
Epoch 8/20
844338/844338 [==============================] - 132s 157us/step - loss: 0.0060 - exp_rmspe_k: 0.0963
Epoch 9/20
844338/844338 [==============================] - 132s 157us/step

In [0]:
SUBMISSION_PATH = os.path.join(HOME_PATH, 'submissions')

In [84]:
SUBMISSION_PATH

'/content/gdrive/My Drive/rossman_project/submissions'

In [0]:

results_list = [pd.read_csv(os.path.join(SUBMISSION_PATH, f'sub__epochs20_batch_size128_{i}.csv'), names=[f'Id_{i}', f'Sales_{i}'], header=0, low_memory=False) for i in range(7)]  

res = pd.concat(results_list, axis=1)

res = res.drop(['Id_1','Id_2','Id_3','Id_4','Id_5','Id_6'], axis=1)
res = res.rename(columns={'Id_0': 'Id'})
res['Sales'] = res.iloc[:,1:].mean(axis=1)

res.loc[:,['Id', 'Sales']].to_csv(os.path.join(HOME_PATH, 'submissions',(f'sub__epochs20_batch_size128_7avg.csv')), index=False) 

 Kaggle score 0.11125. This  reflects the leaderboard final standings on position 35.